In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select  # Select 클래스 import 추가
import pandas as pd
import time
import pyautogui
import pyperclip


options = Options()
# options.add_argument('start-maximized')         # 브라우저를 최대화된 상태로 시작
options.add_experimental_option('detach', True) # 스크립트가 종료된 후에도 브라우저가 닫히지 않도록 설정
# options.add_argument('headless')

driver = webdriver.Chrome(options=options)

driver.get("http://www.kipris.or.kr/khome/main.jsp")

time.sleep(1)
parent_window = driver.current_window_handle
# 모든 창의 핸들 가져오기
all_windows = driver.window_handles
for window in all_windows:
    if window != parent_window:
        driver.switch_to.window(window)         # 팝업 창 핸들로 전환
        driver.close()                          # 팝업 창 닫기 (팝업 창에서 필요한 동작 수행)
        driver.switch_to.window(parent_window)  # 부모 창으로 다시 전환
        break  


login = driver.find_element(By.ID, 'btnLogin')
login.click()
time.sleep(2)

# 로그인 팝업창 선택
driver.switch_to.window(driver.window_handles[1])

# 로그인 ID
login_id = driver.find_element(By.ID, 'login_id')
login_id.send_keys('hwooks.choi@gmail.com')

# 로그인 pw
login_pw = driver.find_element(By.ID, 'login_pw')
login_pw.send_keys('9697003kK*')

# 로그인 버튼
btn_login = driver.find_element(By.ID, 'btn_login')
btn_login.click()

# 5초 기다린 후 팝업 닫기
time.sleep(5)

driver.switch_to.window(parent_window)


try:
    # 특허 클릭
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#gnb > li.cbp-hropen.depth1-1 > div > ul > li:nth-child(1) > a"))
    )
    login_button.click()
    time.sleep(1)

    ## Type Query Sentence
    elem=driver.find_element(By.ID, 'queryText')
    elem.send_keys('("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"딥러닝"+"deep learning")*AD=[20130624~20240624]')
    time.sleep(2)

    # 검색 버튼 클릭
    # <span class="input_btn"><a href="javascript:DoSearch()"><img src="/kpat/images/common/btn_search.gif" alt="검색"></a></span>
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".input_btn"))
    )
    search_button.click()
    time.sleep(1)
    
    select_element = driver.find_element(By.ID,'opt28')         # Select 요소를 찾아서 인스턴스화
    dropdown = Select(select_element)

    dropdown.select_by_value('90')                              # '페이지당 90개' 옵션 선택
    time.sleep(2)
    select_element = driver.find_element(By.CSS_SELECTOR,'#pageSel > a').click()        # 페이지당 특허 건 수 반영
    time.sleep(3)

    # 리스트 형식 변경 (요약 함께보기)
    btnSimple = driver.find_element(By.CSS_SELECTOR, '#btnTextView')
    btnSimple.click()


    # 총 페이지 수 가져오기 
    articles_info = driver.find_element(By.CSS_SELECTOR, 'p.articles').text
    total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())
    # print(f"Total pages: {total_pages}")

    patents_li=[];
    abstracts_li=[];
    priority_dates = []
    international_dates = []
    #for page_number in range(1, total_pages + 1):
    for page_number in range(1, 3):
        # 페이지 번호를 클릭하여 페이지 변경
        if page_number != 1 and page_number % 10 != 0:
            page_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"//span[@class='board_pager03']//a[text()='{page_number}']"))
            )
            page_link.click()
            time.sleep(4)  # 페이지가 로드될 때까지 대기 
        elif page_number % 10 == 0:
            page_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[10]'))
            )
            page_link.click()
            time.sleep(4)  # 페이지가 로드될 때까지 대기 
        # 각 페이지의 특허 제목 추출
        titles = driver.find_elements(By.CSS_SELECTOR, 'h1.stitle')
        abstracts = driver.find_elements(By.CSS_SELECTOR, 'div.search_txt')
                    # 대리인 추출
        # deputies = driver.find_elements(By.CSS_SELECTOR, 'li.left_width.txt_deputy')
        # for index, deputy in enumerate(deputies, start=1):
        #     print(f"대리인 {index}: {deputy.text}")

        # final_rights_holders = driver.find_elements(By.CSS_SELECTOR, 'li.right')
        # for index, final_right_holder in enumerate(final_rights_holders, start=1):
        #     print(f"최종권리자 {index}: {final_right_holder.text}")

        # # 우선권 주장 일자 추출
        # priority_claim_dates = driver.find_elements(By.CSS_SELECTOR, 'ui.search_setList li:nth-child(1)')
        # for index, priority_claim_date in enumerate(priority_claim_dates, start=1):
        #     print(f"우선권 주장 일자 {index}: {priority_claim_date.text}")
        # # 국제 출원 일자 추출
        # international_dates = driver.find_elements(By.CSS_SELECTOR, 'ui.search_setList li:nth-child(2)')
        # for index, international_date in enumerate(international_dates, start=1):
        #     print(f"국제 출원 일자 {index}: {international_date.text}")

            # # 최종권리자 추출
            # final_rights_holder_element = module.find_element(By.CSS_SELECTOR, '.search_info_list .right a')
            # final_rights_holder = final_rights_holder_element.text
            # print("final_rights_holder", final_rights_holder)

            # # 우선권주장일자 추출
            # priority_claim_date_element = module.find_element(By.CSS_SELECTOR, '.search_setList li:nth-child(1)')
            # priority_claim_date = priority_claim_date_element.text.split(':')[1].strip()
            # if not priority_claim_date:
            #     priority_claim_date = None
            # print("priority_claim_date", priority_claim_date)

            # # 국제출원일자 추출
            # international_filing_date_element = module.find_element(By.CSS_SELECTOR, '.search_setList li:nth-child(2)')
            # international_filing_date = international_filing_date_element.text.split(':')[1].strip()
            # if not international_filing_date:
            #     international_filing_date = None
            # print("international_filing_date", international_filing_date)



except Exception as e:
    print('An error occurred:', e)


# finally:
#     driver.quit()
#df=pd.DataFrame(patents_li, columns=['Patent_Titles'])
#df

An error occurred: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=126.0.6478.63); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF751A4EEA2+31554]
	(No symbol) [0x00007FF7519C7ED9]
	(No symbol) [0x00007FF75188872A]
	(No symbol) [0x00007FF75188E55E]
	(No symbol) [0x00007FF751890EB1]
	(No symbol) [0x00007FF751890F50]
	(No symbol) [0x00007FF7518DA48C]
	(No symbol) [0x00007FF7518CC006]
	(No symbol) [0x00007FF7518FD02A]
	(No symbol) [0x00007FF7518CBA76]
	(No symbol) [0x00007FF7518FD240]
	(No symbol) [0x00007FF75191C977]
	(No symbol) [0x00007FF7518FCDD3]
	(No symbol) [0x00007FF7518CA33B]
	(No symbol) [0x00007FF7518CAED1]
	GetHandleVerifier [0x00007FF751D58B1D+3217341]
	GetHandleVerifier [0x00007FF751DA5AE3+3532675]
	GetHandleVerifier [0x00007FF751D9B0E0+3489152]
	GetHandleVerifier [0x00007FF75

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select  # Select 클래스 import 추가
import pandas as pd
import time
import pyautogui
import pyperclip


options = Options()
# options.add_argument('start-maximized')         # 브라우저를 최대화된 상태로 시작
options.add_experimental_option('detach', True) # 스크립트가 종료된 후에도 브라우저가 닫히지 않도록 설정
# options.add_argument('headless')

driver = webdriver.Chrome(options=options)

driver.get("http://www.kipris.or.kr/khome/main.jsp")

time.sleep(1)
parent_window = driver.current_window_handle
# 모든 창의 핸들 가져오기
all_windows = driver.window_handles
for window in all_windows:
    if window != parent_window:
        driver.switch_to.window(window)         # 팝업 창 핸들로 전환
        driver.close()                          # 팝업 창 닫기 (팝업 창에서 필요한 동작 수행)
        driver.switch_to.window(parent_window)  # 부모 창으로 다시 전환
        break  


login = driver.find_element(By.ID, 'btnLogin')
login.click()
time.sleep(2)

# 로그인 팝업창 선택
driver.switch_to.window(driver.window_handles[1])

# 로그인 ID
login_id = driver.find_element(By.ID, 'login_id')
login_id.send_keys('hwooks.choi@gmail.com')

# 로그인 pw
login_pw = driver.find_element(By.ID, 'login_pw')
login_pw.send_keys('9697003kK*')

# 로그인 버튼
btn_login = driver.find_element(By.ID, 'btn_login')
btn_login.click()

# 5초 기다린 후 팝업 닫기
time.sleep(5)

driver.switch_to.window(parent_window)


try:
    # 특허 클릭
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#gnb > li.cbp-hropen.depth1-1 > div > ul > li:nth-child(1) > a"))
    )
    login_button.click()
    time.sleep(1)

    ## Type Query Sentence
    elem=driver.find_element(By.ID, 'queryText')
    elem.send_keys('("AI"+"artificial intelligence"+"인공지능"+"인공 지능"+"딥러닝"+"deep learning")*AD=[20130624~20240624]')
    time.sleep(2)

    # 검색 버튼 클릭
    # <span class="input_btn"><a href="javascript:DoSearch()"><img src="/kpat/images/common/btn_search.gif" alt="검색"></a></span>
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".input_btn"))
    )
    search_button.click()
    time.sleep(1)
    
    select_element = driver.find_element(By.ID,'opt28')         # Select 요소를 찾아서 인스턴스화
    dropdown = Select(select_element)

    dropdown.select_by_value('90')                              # '페이지당 90개' 옵션 선택
    time.sleep(2)
    select_element = driver.find_element(By.CSS_SELECTOR,'#pageSel > a').click()        # 페이지당 특허 건 수 반영
    time.sleep(3)

    # 리스트 형식 변경 (요약 함께보기)
    btnSimple = driver.find_element(By.CSS_SELECTOR, '#btnTextView')
    btnSimple.click()


    # 총 페이지 수 가져오기 
    articles_info = driver.find_element(By.CSS_SELECTOR, 'p.articles').text
    total_pages = int(articles_info.split('/')[-1].replace(',', '').replace('Pages)', '').strip())
    # print(f"Total pages: {total_pages}")

    patents_li=[];
    abstracts_li=[];
    priority_dates = []
    international_dates = []
    #for page_number in range(1, total_pages + 1):
    for page_number in range(1, 3):
        # 페이지 번호를 클릭하여 페이지 변경
        if page_number != 1 and page_number % 10 != 0:
            page_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"//span[@class='board_pager03']//a[text()='{page_number}']"))
            )
            page_link.click()
            time.sleep(4)  # 페이지가 로드될 때까지 대기 
        elif page_number % 10 == 0:
            page_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="divMainArticle"]/form/section/div[2]/span/a[10]'))
            )
            page_link.click()
            time.sleep(4)  # 페이지가 로드될 때까지 대기 
        # 각 페이지의 특허 제목 추출
        # titles = driver.find_elements(By.CSS_SELECTOR, 'h1.stitle')
        # abstracts = driver.find_elements(By.CSS_SELECTOR, 'div.search_txt')
        #             # 대리인 추출
        # deputies = driver.find_elements(By.CSS_SELECTOR, 'li.left_width.txt_deputy')
        # for index, deputy in enumerate(deputies, start=1):
        #     print(f"대리인 {index}: {deputy.text}")

        # deputies = driver.find_elements(By.CSS_SELECTOR, 'li.left_width.txt_deputy')
        # for index, deputy in enumerate(deputies, start=1):
        #     print(f"대리인 {index}: {deputy.text}")

        final_rights_holders = driver.find_elements(By.CSS_SELECTOR, 'li.right')
        for index, final_right_holder in enumerate(final_rights_holders, start=1):
            print(f"최종권리자 {index}: {final_right_holder.text}")

        # 우선권 주장 일자 추출
        priority_claim_dates = module.find_elements(By.CSS_SELECTOR, 'ui.search_setList li:nth-child(1)')
        for index, priority_claim_date in enumerate(priority_claim_dates, start=1):
            print(f"우선권 주장 일자 {index}: {priority_claim_date.text}")
        # 국제 출원 일자 추출
        international_dates = module.find_elements(By.CSS_SELECTOR, 'ui.search_setList li:nth-child(2)')
        for index, international_date in enumerate(international_dates, start=1):
            print(f"국제 출원 일자 {index}: {international_date.text}")
        


            # # 최종권리자 추출
            # final_rights_holder_element = module.find_element(By.CSS_SELECTOR, '.search_info_list .right a')
            # final_rights_holder = final_rights_holder_element.text
            # print("final_rights_holder", final_rights_holder)

            # # 우선권주장일자 추출
            # priority_claim_date_element = module.find_element(By.CSS_SELECTOR, '.search_setList li:nth-child(1)')
            # priority_claim_date = priority_claim_date_element.text.split(':')[1].strip()
            # if not priority_claim_date:
            #     priority_claim_date = None
            # print("priority_claim_date", priority_claim_date)

            # # 국제출원일자 추출
            # international_filing_date_element = module.find_element(By.CSS_SELECTOR, '.search_setList li:nth-child(2)')
            # international_filing_date = international_filing_date_element.text.split(':')[1].strip()
            # if not international_filing_date:
            #     international_filing_date = None
            # print("international_filing_date", international_filing_date)










        time.sleep(4)  # 페이지가 로드될 때까지 대기 

        #modules = driver.find_elements(By.CSS_SELECTOR, 'div.search_basic_info.noimg_list')
        #for index, module in enumerate(modules, start=1):
        #     # 출원인 추출
        #     applicant_element = module.find_element(By.CSS_SELECTOR, '.search_info_list .left_width.letter1 a')
        #     applicant = applicant_element.text
        #     print("applicant", applicant)

        #     # 출원일자 추출
        #     application_date_element = module.find_element(By.CSS_SELECTOR, '.search_info_list .right_width:nth-of-type(2)')
        #     application_date = application_date_element.text.split(':')[1].strip()
        #     print("application_date", application_date)

        #     # 출원번호 추출
        #     application_number_element = module.find_element(By.CSS_SELECTOR, '#divRealContent2 > ul > li:nth-child(2) > span:nth-child(1)')
        #     application_number = application_number_element.text
        #     print("application_number", application_number)

        #     # # 대리인 추출
        #     # deputy_element = module.find_element(By.CSS_SELECTOR, 'li.left_width.txt_deputy')
        #     # deputy = deputy_element.text
        #     # print("deputy", deputy)

            # # 최종권리자 추출
            # final_rights_holder_element = module.find_element(By.CSS_SELECTOR, '.search_info_list .right a')
            # final_rights_holder = final_rights_holder_element.text
            # print("final_rights_holder", final_rights_holder)

            # # 우선권주장일자 추출
            # priority_claim_date_element = module.find_element(By.CSS_SELECTOR, '.search_setList li:nth-child(1)')
            # priority_claim_date = priority_claim_date_element.text.split(':')[1].strip()
            # if not priority_claim_date:
            #     priority_claim_date = None
            # print("priority_claim_date", priority_claim_date)

            # # 국제출원일자 추출
            # international_filing_date_element = module.find_element(By.CSS_SELECTOR, '.search_setList li:nth-child(2)')
            # international_filing_date = international_filing_date_element.text.split(':')[1].strip()
            # if not international_filing_date:
            #     international_filing_date = None
            # print("international_filing_date", international_filing_date)


        # for index, title in enumerate(titles, start=1):
        #     print(f"Title {index}: {title.text}"); patents_li.append(title.text)
        # for index, abstract in enumerate(abstracts, start=1):
        #     print(f"abstract {index}: {abstract.text}"); abstracts_li.append(abstract.text)


except Exception as e:
    print('An error occurred:', e)


# finally:
#     driver.quit()
#df=pd.DataFrame(patents_li, columns=['Patent_Titles'])
#df

An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.63)
Stacktrace:
	GetHandleVerifier [0x00007FF751A4EEA2+31554]
	(No symbol) [0x00007FF7519C7ED9]
	(No symbol) [0x00007FF75188872A]
	(No symbol) [0x00007FF75185D995]
	(No symbol) [0x00007FF7519044D7]
	(No symbol) [0x00007FF75191C051]
	(No symbol) [0x00007FF7518FCDD3]
	(No symbol) [0x00007FF7518CA33B]
	(No symbol) [0x00007FF7518CAED1]
	GetHandleVerifier [0x00007FF751D58B1D+3217341]
	GetHandleVerifier [0x00007FF751DA5AE3+3532675]
	GetHandleVerifier [0x00007FF751D9B0E0+3489152]
	GetHandleVerifier [0x00007FF751AFE776+750614]
	(No symbol) [0x00007FF7519D375F]
	(No symbol) [0x00007FF7519CEB14]
	(No symbol) [0x00007FF7519CECA2]
	(No symbol) [0x00007FF7519BE16F]
	BaseThreadInitThunk [0x00007FFC09757344+20]
	RtlUserThreadStart [0x00007FFC0B23CC91+33]

